In [1]:
import json
import csv
import numpy as np
import pandas as pd
from os import path, mkdir
import glob
from datetime import datetime, timedelta, date
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
bfh=pd.read_csv('c:/dev/RP/server/data/horses/horses_2020-01-01.csv')
bfh=bfh.loc[bfh['rid']==10312]
bfp=pd.read_csv('c:/dev/RP/server/data/bf/changes_2020-01-01.csv')
bfp['dt']=pd.to_datetime(bfp['dt'])
bfp=bfp.loc[bfp['mid']==166899509]

In [11]:
bfp['mean']=bfp.groupby(['rid'])['price'].transform('mean')

In [15]:
bfp['d_']=(bfp['price']-bfp['mean'])/bfp['mean']
bfp['d2']=bfp['d1']/bfp['mean']

In [16]:
bfp

,mid,dt,rid,price,mean,d1,d2
434,166899509,2019-12-31 15:59:58,24545075,0.208333,0.322869,-0.114535,-0.354743
451,166899509,2019-12-31 16:01:01,24545075,0.200000,0.322869,-0.122869,-0.380553
636,166899509,2019-12-31 16:28:52,7369339,0.125000,0.078032,0.046968,0.601905
903,166899509,2019-12-31 16:59:32,26725344,0.040000,0.120720,-0.080720,-0.668655
1036,166899509,2019-12-31 17:08:54,24545075,0.277778,0.322869,-0.045091,-0.139657
...,...,...,...,...,...,...,...
48849,166899509,2020-01-01 15:00:01,13564829,0.001695,0.257774,-0.256079,-0.993425
48850,166899509,2020-01-01 15:00:01,26965594,0.001000,0.005932,-0.004932,-0.831414
48851,166899509,2020-01-01 15:00:01,26725344,0.001000,0.120720,-0.119720,-0.991716
48852,166899509,2020-01-01 15:00:01,9570145,0.001000,0.025417,-0.024417,-0.960657


# HElo

In [3]:
n=5
m=16
losers=[-x for x in range(1,m-n+1)]
s=sum(losers)*-1
winners=[2**x for x in range(0,n)]
price=(s-1)/sum(winners[1:])
profits=[x*price for x in winners[1:]]
res=losers[::-1]+[1]+profits
print(res,sum(res))

[-11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 1, 4.333333333333333, 8.666666666666666, 17.333333333333332, 34.666666666666664] -7.105427357601002e-15


# Concat to one table for analysis

In [12]:
def parse_dt(x):
    h,m=map(int,x[0].split(':'))
    ds = x[1]
    hours_add=11 if (ds>=datetime(2019,3,31,0,0,0) and ds<datetime(2019,10,27,0,0,0)) or (ds>=datetime(2018,3,25,0,0,0) and ds<datetime(2018,10,28,0,0,0)) or (ds>=datetime(2017,3,26,0,0,0) and ds<datetime(2017,10,29,0,0,0)) or (ds>=datetime(2016,3,27,0,0,0) and ds<datetime(2016,10,30,0,0,0)) or ds<datetime(2015,10,25,0,0,0) else 12
    if h<9:
        h+=hours_add
    
    ds+= timedelta(hours=h)
    ds+= timedelta(minutes=m)
    return ds

def convert_distance(txt):
    f=txt.replace('½','.5').replace('f','')
    dm=0
    if 'm' in f:
        m,f=f.split('m')
        dm=float(m)*1609
    df=0 if f=='' else float(f)*201
    return dm+df

def horses_count(id):
    return len(df_horses[df_horses['rid']==id].index)

def convert_condition(df):
    dic={'Standard':0, 'Good':1, 'Good To Firm':2, 'Very Soft':3,
       'Good To Yielding':4, 'Soft':5, 'Yielding':6, 'Fast':7, 'Firm':8, 'Heavy':9,
       'Good To Soft':10, 'Yielding To Soft':11, 'Soft To Heavy':12,
       'Standard To Slow':13,'Standard To Fast':14, 'Sloppy':15, 'Muddy':16, 'Slow':17,'Holding':18, 'Frozen':19,'Abandoned':20, np.NaN:0}
    return df['condition'].replace(dic)

def convert_class(df):
    df['rclass']=df['rclass'].fillna('')
    dic={np.NaN:0, '':0, 'Class 1':1, 'Class 2':2, 'Class 3':3,
       'Class 4':4, 'Class 5':5, 'Class 6':6, 'Class 7':7}
    return df['rclass'].replace(dic)

def convert_country(df):
    df['countryCode']=df['course'].apply(lambda x: x.split('(')[-1].replace(')',''))
    df['countryCode']=np.where(df['countryCode'].str.len()>4,'GB',df['countryCode'])
    dic={'AW':'GB', 'York':'GB', 'July':'GB', 'Bath':'GB', 'IRE':'IE', 'USA':'US','AUS':'AU','Ayr':'GB','JPN':'JP','UAE':'AE','ITY':'IT'}
    return df['countryCode'].replace(dic)

In [5]:
def convert_races(df_races):
    df_races['metric']=df_races['distance'].apply(lambda x: convert_distance(x))
    df_races['countryCode']=convert_country(df_races)
    df_races['marketTime']=df_races.apply(lambda x: parse_dt(x[['time','date']]), axis=1)
    df_races['ncond']=convert_condition(df_races)
    df_races['marketTime']=pd.to_datetime(df_races['marketTime'])
    df_races['class']=convert_class(df_races)
    return df_races

def convert_horses(df_horses):
    df_horses.loc[df_horses['decimalPrice']>=4600000000000000,'decimalPrice']=4.06
    df_horses.loc[df_horses['decimalPrice']>=3833333333333330,'decimalPrice']=3.08
    df_horses.loc[df_horses['decimalPrice']>=1909090909090900,'decimalPrice']=12
    df_horses.loc[df_horses['decimalPrice']>=1833333333333330,'decimalPrice']=13
    df_horses.loc[df_horses['decimalPrice']>=1714285714285710,'decimalPrice']=15
    df_horses.loc[df_horses['decimalPrice']>=1357142857142850,'decimalPrice']=29
    df_horses.loc[df_horses['decimalPrice']>=1303030303030300,'decimalPrice']=1.03
    df_horses.loc[df_horses['decimalPrice']>=1625,'decimalPrice']=17
    df_horses['runners']=df_horses.groupby(['rid'])['horseName'].transform('count')
    df_horses=df_horses.loc[df_horses['runners']>1]
    dic=['DSQ','PU', 'F', 'UR', 'RR', 'RO', 'SU', 'REF', 'BD', 'CO', 'LFT', '0']
    df_horses.loc[df_horses['position'].isin(dic),'position']='40'
    df_horses['position']=df_horses['position'].astype(int)
    df_horses['decimalPrice']=1/df_horses['decimalPrice']
    df_horses['margin']=df_horses.groupby(['rid'])['decimalPrice'].transform('sum')
    df_horses=df_horses.loc[df_horses['margin']>1]
    df_horses['weight']=(df_horses['weightSt']*6.35+df_horses['weightLb']*0.454).astype(int)
    #df_horses['win_drift']=(df_horses['win_close']-df_horses['win_mean'])/df_horses['win_mean']
    df_horses.loc[df_horses['position']==1,'res_win']=1
    df_horses.loc[df_horses['position']!=1,'res_win']=0
    df_horses.loc[(df_horses['position']==1),'res_place']=1
    df_horses.loc[(df_horses['position']==2) & (df_horses['runners']>=5),'res_place']=1 # 5,6,7 runners 
    df_horses.loc[(df_horses['position']==3) & (df_horses['runners']>=8),'res_place']=1 # 8 to 15
    df_horses.loc[(df_horses['position']==4) & (df_horses['runners']>=16),'res_place']=1 # 16+
    df_horses.loc[(df_horses['position']>1) & (df_horses['runners']<5),'res_place']=0 # <5 runners 
    df_horses.loc[(df_horses['position']>2) & (df_horses['runners']>=5),'res_place']=0 # 5,6,7 runners 
    df_horses.loc[(df_horses['position']>3) & (df_horses['runners']>=8),'res_place']=0 # 8 to 15
    df_horses.loc[(df_horses['position']>4) & (df_horses['runners']>=16),'res_place']=0 # 16+
    df_horses=df_horses.merge(df_races[['rid','course','ncond','metric','class','marketTime']], on='rid', how='left')
    df_horses=df_horses.sort_values(by=['marketTime'])
    # Races averages
    # Danger: position_diff, position_rank
    cols=['age', 'decimalPrice', 'weight', 'RPR', 'TR', 'OR', 'position']
    for col in cols:
        print(col)
        df_horses[col+'_mean']=df_horses.groupby(['rid'])[col].transform('mean')
        df_horses[col+'_diff']=df_horses[col]-df_horses[col+'_mean']
        df_horses[col+'_diff'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
        df_horses[col+'_rank']=df_horses.groupby(['rid'])[col+'_diff'].rank(method='dense', ascending=False).astype(int)
        df_horses.drop(columns=[col+'_mean'], inplace=True)
    return df_horses


In [36]:
races_cols=['rid','course','time','date','title','rclass','band','ages','distance','condition','hurdles','prizes','currency','winningTime','link','performer','performerNotes','eyecatcher','eyecatcherNotes']
horses_cols=['rid','horseName','horseLink','age','saddle','price','decimalPrice','isFav','trainerLink','trainerName','jockeyLink','jockeyName','position','positionL','dist','weightSt','weightLb','overWeight','outHandicap','headGear','RPR','TR','OR','father','mother','gfather','fatherlink','motherlink','gfatherlink','comment']
li=[]
for filename in glob.glob(os.path.join('data/done/', "races_*.csv")):
    df = pd.read_csv(filename, index_col=None, header=None)
    df.columns=races_cols
    li.append(df)
df_races = pd.concat(li, axis=0, ignore_index=True)
li=[]
for filename in glob.glob(os.path.join('data/done/', "horses_*.csv")):
    df = pd.read_csv(filename, index_col=None, header=None)
    df.columns=horses_cols
    li.append(df)
df_horses = pd.concat(li, axis=0, ignore_index=True)
#df_races=pd.concat([pd.read_csv('data/races_all.csv'), pd.read_csv('data/races.csv')], ignore_index=True)
#df_races=pd.read_csv('data/races_all.csv', index_col=False)
#df_horses=pd.read_csv('data/horses_all.csv', index_col=False)
# Keep races from 2010
df_races['date']=pd.to_datetime(df_races['date'])
#ds = datetime.strptime('2010-01-01', '%Y-%m-%d')
#df_races=df_races[df_races['date']>=ds]

# Filter out horses
#df_horses=df_horses[df_horses['rid'].isin(df_races['rid'])]
#df_horses.reset_index(drop=True, inplace=True)

rids_noprice=df_horses.loc[df_horses['decimalPrice'].isna(),'rid'].unique()
df_races=df_races[~df_races['rid'].isin(rids_noprice)]
df_horses=df_horses[~df_horses['rid'].isin(rids_noprice)]

df_races=convert_races(df_races)
df_horses=convert_horses(df_horses)
df_horses_all=pd.read_csv('data/horses_from2010.csv')[df_horses.columns]
df_horses=pd.concat([df_horses_all,df_horses], axis=0, ignore_index=True)
df_horses['marketTime']=pd.to_datetime(df_horses['marketTime'])
df_horses=df_horses.sort_values(by=['marketTime'])
df_bak=df_horses.copy()
df_horses.to_csv('data/horses_from2010.csv')

age
decimalPrice
weight
RPR
TR
OR
position


In [ ]:
df_horses=df_bak.copy()

## HTJ Cumulatives
### Includes cumulative stats with current

In [37]:
df_horses['N']=1
#cols=['metric', 'res_win', 'res_place', 'decimalPrice_diff', 'RPR_diff', 'TR_diff', 'OR_diff', 'win_drift', 'position_diff']
cols=['metric', 'res_win', 'res_place', 'decimalPrice_diff', 'RPR_diff', 'TR_diff', 'OR_diff', 'position_diff']
# Horses
group_feature='horseName'
df_horses['C']=df_horses.groupby([group_feature])['N'].cumsum()#-1
for col in cols:
    df_horses[col+'_h_sum']=df_horses.groupby([group_feature])[col].cumsum()#-df_horses[col]
    df_horses[col+'_h_avg']=df_horses[col+'_h_sum']/df_horses['C']

# Trainers
group_feature='trainerName'
df_horses['C']=df_horses.groupby([group_feature])['N'].cumsum()#-1
for col in cols:
    df_horses[col+'_t_sum']=df_horses.groupby([group_feature])[col].cumsum()#-df_horses[col]
    df_horses[col+'_t_avg']=df_horses[col+'_t_sum']/df_horses['C']


# Jockeys
group_feature='jockeyName'
df_horses['C']=df_horses.groupby([group_feature])['N'].cumsum()#-1
for col in cols:
    df_horses[col+'_j_sum']=df_horses.groupby([group_feature])[col].cumsum()#-df_horses[col]
    df_horses[col+'_j_avg']=df_horses[col+'_j_sum']/df_horses['C']

## Races averages over Cumulatives

In [38]:
# Disperse of current distance vs always horse's distance
df_horses['metric_abs']=(df_horses['metric']-df_horses['metric_h_avg']).abs()
# Mean disperse through race
df_horses['metric_abs_mean']=df_horses.groupby(['rid'])['metric_abs'].transform('mean')
df_horses['metric_abs_mean'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
df_horses['metric_abs_mean_rank']=df_horses.groupby(['rid'])['metric_abs_mean'].rank(method='dense', ascending=False).astype(int)

#cols=['res_win', 'res_place', 'decimalPrice_diff', 'win_drift', 'position_diff']
cols=['res_win', 'res_place', 'decimalPrice_diff', 'position_diff']
cums=[x+'_h_avg' for x in cols]
for col in cums:
    print(col)
    df_horses[col+'_mean']=df_horses.groupby(['rid'])[col].transform('mean')
    df_horses[col+'_diff']=df_horses[col]-df_horses[col+'_mean']
    df_horses[col+'_diff'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
    df_horses[col+'_rank']=df_horses.groupby(['rid'])[col+'_diff'].rank(method='dense', ascending=False).astype(int)

cums=[x+'_t_avg' for x in cols]
for col in cums:
    print(col)
    df_horses[col+'_mean']=df_horses.groupby(['rid'])[col].transform('mean')
    df_horses[col+'_diff']=df_horses[col]-df_horses[col+'_mean']
    df_horses[col+'_diff'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
    df_horses[col+'_rank']=df_horses.groupby(['rid'])[col+'_diff'].rank(method='dense', ascending=False).astype(int)

cums=[x+'_j_avg' for x in cols]
for col in cums:
    print(col)
    df_horses[col+'_mean']=df_horses.groupby(['rid'])[col].transform('mean')
    df_horses[col+'_diff']=df_horses[col]-df_horses[col+'_mean']
    df_horses[col+'_diff'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
    df_horses[col+'_rank']=df_horses.groupby(['rid'])[col+'_diff'].rank(method='dense', ascending=False).astype(int)
    

res_win_h_avg
res_place_h_avg
decimalPrice_diff_h_avg
position_diff_h_avg
res_win_t_avg
res_place_t_avg
decimalPrice_diff_t_avg
position_diff_t_avg
res_win_j_avg
res_place_j_avg
decimalPrice_diff_j_avg
position_diff_j_avg


[col for col in df_horses.columns if not '_sum' in col]

In [39]:
#cols_base=['rid','course', 'marketTime', 'horseName', 'position', 'res_win', 'res_place', 'runners', 'ncond', 'metric', 'class', 'decimalPrice', 'age', 'RPR', 'TR', 'OR', 'weight', 'win_drift']
cols_base=['rid','course', 'marketTime', 'horseName', 'position', 'res_win', 'res_place', 'runners', 'ncond', 'metric', 'class', 'decimalPrice', 'age',  'RPR', 'TR', 'OR', 'weight']
cols=[col for col in df_horses.columns if not col in cols_base]

# Ranks
cols_ranks=[col for col in cols if '_rank' in col]
cols=[col for col in cols if not col in cols_ranks]
#cols_ranks=[col for col in cols_ranks if col!='position_rank'] # Remove dangerous poison

# Average of previous races for every horse, trainer and jockey
cols_avg=[col for col in cols if '_avg' in col]
cols=[col for col in cols if not col in cols_avg]

# Diffs from mean inside every racing
cols_diffs=[col for col in cols if '_diff' in col]
cols=[col for col in cols if not col in cols_diffs]
#cols_diffs=[col for col in cols_diffs if col!='position_diff'] # Remove dangerous poison

# The rest is rabbish
print(cols)
cols=cols_base+cols_ranks+cols_avg+cols_diffs
cols=[col for col in cols if not '_mean' in col and not '_sum' in col]
df_horses=df_horses[cols]
df_horses.to_csv('data/horses_stats.csv')

['horseLink', 'saddle', 'price', 'isFav', 'trainerLink', 'trainerName', 'jockeyLink', 'jockeyName', 'positionL', 'dist', 'weightSt', 'weightLb', 'overWeight', 'outHandicap', 'headGear', 'father', 'mother', 'gfather', 'fatherlink', 'motherlink', 'gfatherlink', 'comment', 'margin', 'N', 'C', 'metric_h_sum', 'res_win_h_sum', 'res_place_h_sum', 'metric_t_sum', 'res_win_t_sum', 'res_place_t_sum', 'metric_j_sum', 'res_win_j_sum', 'res_place_j_sum', 'metric_abs', 'metric_abs_mean']


In [32]:
df_horses.to_csv('data/horses_stats.csv')

# Concat days

In [8]:
df_stat_all=pd.read_csv('data/horses_stats.csv', index_col=False)
df_stat_all['hname']=df_stat_all['horseName'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')
df_stat_all['marketTime']=pd.to_datetime(df_stat_all['marketTime'])
#df_stat_all=df_stat_all.sort_values(by=['marketTime'])

df_horses=pd.read_csv('data/horses_from2015.csv', index_col=False)
df_horses['marketTime']=pd.to_datetime(df_horses['marketTime'])
df_horses['hname']=df_horses['horseName'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')

In [57]:
cols_res=['position','res_win', 'res_place']
cols_home=['course', 'marketTime', 'horseName','hname', 'runners', 'ncond', 'metric', 'class', 'decimalPrice', 'weight','age','RPR', 'TR', 'OR']
cols_cur_diffs=['age_diff','age_rank', 'decimalPrice_diff', 'decimalPrice_rank', 'weight_diff','weight_rank', 'RPR_diff', 'RPR_rank', 'TR_diff', 'TR_rank', 'OR_diff','OR_rank']
cols_horses=cols_res+cols_home+cols_cur_diffs

cols_stats=['hname','res_win_h_avg_rank', 'res_place_h_avg_rank','decimalPrice_diff_h_avg_rank', 'position_diff_h_avg_rank','res_win_t_avg_rank', 'res_place_t_avg_rank','decimalPrice_diff_t_avg_rank', 'position_diff_t_avg_rank','res_win_j_avg_rank', 'res_place_j_avg_rank','decimalPrice_diff_j_avg_rank', 'position_diff_j_avg_rank','metric_h_avg', 'res_win_h_avg', 'res_place_h_avg','decimalPrice_diff_h_avg', 'RPR_diff_h_avg', 'TR_diff_h_avg','OR_diff_h_avg', 'position_diff_h_avg', 'metric_t_avg', 'res_win_t_avg','res_place_t_avg', 'decimalPrice_diff_t_avg', 'RPR_diff_t_avg','TR_diff_t_avg', 'OR_diff_t_avg', 'position_diff_t_avg', 'metric_j_avg','res_win_j_avg', 'res_place_j_avg', 'decimalPrice_diff_j_avg','RPR_diff_j_avg', 'TR_diff_j_avg', 'OR_diff_j_avg','position_diff_j_avg', 'res_win_h_avg_diff', 'res_place_h_avg_diff','decimalPrice_diff_h_avg_diff', 'position_diff_h_avg_diff','res_win_t_avg_diff', 'res_place_t_avg_diff','decimalPrice_diff_t_avg_diff', 'position_diff_t_avg_diff','res_win_j_avg_diff', 'res_place_j_avg_diff','decimalPrice_diff_j_avg_diff', 'position_diff_j_avg_diff']

def make_day(d):
    df_stat=df_stat_all.loc[df_stat_all['marketTime']<d]
    df_stat.drop_duplicates(subset=['hname'], keep='last', inplace=True)
    df=df_horses.loc[(df_horses['marketTime']>=d) & (df_horses['marketTime'] < d + timedelta(days=1))]
    return df[cols_horses].merge(df_stat[cols_stats], on='hname', how='left')


In [ ]:
ds = datetime.strptime('2015-01-01', '%Y-%m-%d')
de = datetime.strptime('2020-01-03', '%Y-%m-%d')
#de = datetime.strptime('2020-04-09', '%Y-%m-%d')
d = ds
while d <= de:
    print(d)
    df=make_day(d)
    if not df.empty:
        if d>ds:
            df_res=pd.concat([df_res,df], ignore_index=True)
        else:
            df_res=make_day(d)
    d+=timedelta(days=1)
df_res.to_csv('data/to_train.csv')

In [ ]:
df_today = pd.concat(map(pd.read_csv, glob.glob(path.join('data/today/', '*.csv'))))
df_today=df_today.reset_index(drop=True)
df_today['rclass']=df_today['rclass'].fillna('')
df=df_today[cols_home]
df['class']=convert_class(df_today)
df['rid']=df_today.index
df['rid']=df.groupby(['course','marketTime'])['rid'].transform('sum')
for col in cols_null:
    df[col]=0
df['ncond']=convert_condition(df_today)
df['runners']=df.groupby(['course','marketTime'])['horseName'].transform('count')
df['metric']=df_today['distance'].apply(lambda x: convert_distance(x))
df['decimalPrice']=1/df['decimalPrice']
df['weight']=(df_today['weightSt']*6.35+df_today['weightLb']*0.454).astype(int)

for col in cols_home_diffs_ranks:
    print(col)
    df[col+'_mean']=df.groupby(['rid'])[col].transform('mean')
    df[col+'_diff']=df[col]-df[col+'_mean']
    df[col+'_diff'].replace([np.inf, -np.inf,np.nan], 0,inplace=True)
    df[col+'_rank']=df.groupby(['rid'])[col+'_diff'].rank(method='dense', ascending=False).astype(int)

df.merge(df_stat[cols_stats], on='hname', how='left').to_csv('data/topred.csv')

# Others

In [2]:
from selenium import webdriver
from os import path, listdir
import shutil
import time
import re

In [6]:
def load_today():
    driver = webdriver.Chrome(executable_path=r'../../lib/chromedriver.exe')
    load_atr(driver)
    #load_atr(None)
    load_rp(driver)
    driver.close()

def load_atr(driver):
    fn = 'output/today/atr.html'
    driver.get('https://www.attheraces.com/racecards')
    time.sleep(5)
    html = driver.page_source
    #with open(fn, 'r', encoding='utf8') as f:
    #    html=f.read()
    parse_atr_day(html, driver)
    with open(fn, 'w+', encoding='utf8') as f:
        f.write(html)
    print('ATR: {} done with {} bytes.'.format(fn, len(html)))

def load_rp(driver):
    fn = 'output/today/rp.html'
    driver.get('https://www.racingpost.com/racecards/')
    time.sleep(5)
    html = driver.page_source
    parse_rp_day(html, driver)
    with open(fn, 'w+', encoding='utf8') as f:
        f.write(html)
    print('RP: {} done with {} bytes.'.format(fn, len(html)))

pRPLinks = re.compile(r'data-racecourse="[^"]+" href="([^"]+)">')
def parse_rp_day(html, driver):
    links = pRPLinks.findall(html)
    for link in links:
        fn = 'output/today/rp/{}.html'.format(link.split('/')[-1])
        load_link(f'https://www.racingpost.com{link}', fn, driver)


pATR_places = re.compile(r'<span class="h6 visible">([^<]+)')
pATR_races = re.compile(r'<span class="h7">([^<]+)')
pATR_split='<span class="h6 visible">'
pATR_Links = re.compile(r'<a href="([^"]+)"[^>]+>\s*<span class="h7">')
def parse_atr_day(html, driver):
    races=[]
    places=pATR_places.findall(html)
    parts=html.split(pATR_split)[1:]
    for i,part in enumerate(parts):
        if 'Abandoned' in part:
            continue
        place=places[i].strip()
        races = pATR_races.findall(part)
        links = pATR_Links.findall(part)
        for j,race in enumerate(races):
            ps=race.split(' - ')
            name=ps[-1].strip()
            tstr=ps[0].strip()
            tm=get_date(tstr)
            if 'Suspended' in name:# or (tm-timedelta(minutes=5))<datetime.now():
                continue
            link=links[j].strip()
            fn=f'output/today/atr/{name}_{tstr}.html'.replace(':','')
            load_link('https://www.attheraces.com'+link, fn, driver)
            print(tm, place, tstr, name, link)

def getone(pattern, string):
    m = pattern.search(string)
    res = m.group(1) if m else ''
    return res.strip()

def get_date(tstr):
    t = datetime.strptime('{:%Y-%m-%d} {}'.format(date.today(),tstr), '%Y-%m-%d %H:%M')
    return t+timedelta(hours=6)

def load_link(link, fn, driver):
    if not (path.exists(fn) and path.getsize(fn) > 10000):
        driver.get(link)
        time.sleep(10)
        html = driver.page_source
        with open(fn, 'w+', encoding='utf8') as f:
            f.write(html)
        print('{} done with {} bytes.'.format(fn, len(html)))

In [31]:
def parse_today():
    folder='output/today/atr/'
    files=listdir(folder)
    for fn in files:
        with open(folder+fn, 'r', encoding='utf8') as f:
            html = f.read()
            print('parse '+fn)
            parse_atr(html)
        fn_target=folder.replace('today','done')+'{:%Y-%m-%d}_'.format(date.today())+fn
        shutil.move(folder+fn, fn_target)
    folder='output/today/rp/'
    files=listdir(folder)
    for fn in files:
        with open(folder+fn, 'r', encoding='utf8') as f:
            html = f.read()
            print('parse '+fn)
            parse_rp(html)
        fn_target=folder.replace('today','done')+'{:%Y-%m-%d}_'.format(date.today())+fn
        shutil.move(folder+fn, fn_target)
    

patrCourse = re.compile(r'<title>([^<]+)')
patrDistance = re.compile(r'<div class="p--large font-weight--semibold">([^<]+)')
patrCondition = re.compile(r'<p class="p--medium">([^<]+)</p>')
patrOR = re.compile(r'<span class="text-pill text-pill--steel tooltip tooltipstered"[^>]+>([^<]+)')
patrHorseName = re.compile(r'<a href="/form/horse/[^>]+>([^<]+)')
patrOddsHorse = re.compile(r'class="odds-grid-horse__name a--plain js-popup-trigger" data-cookie="popupForm">([^<]+)')
patrOddsBookies = re.compile(r'<b class="bookmaker-logo__inner"[^>]+>([^<]+)')
patrOdds = re.compile(r'<span class="odds-value odds-value--decimal">([^<]+)')

def parse_atr(html):
    course=getone(patrCourse, html)
    ps=course.split(' | ')
    t=ps[0]
    course=ps[1]
    condition=getone(patrCondition, html).split('(')[0].replace('&nbsp;','')
    distance=getone(patrDistance, html)
    parts=html.split('<h2 class="h6 horse__details flush">')[1:]
    prices_count=html.count('<div class="odds-grid__cell odds-grid__cell--ew">')
    horses=[]
    for part in parts:
        if 'Non Runner' in part:
            continue
        horse=getone(patrHorseName, part)
        OR=getone(patrOR, part).replace('&nbsp;-&nbsp;','')
        horses.append([course, t, condition, distance, horse,OR])
    oddsHorses = patrOddsHorse.findall(html)
    oddsBookies = patrOddsBookies.findall(html)[:-1]
    oddsOdds=[]
    prices=patrOdds.findall(html)
    for x in prices:
        oddsOdds.append(0 if x=='-' or x=='odds' or x=='SP' or x=='N/A' else float(x))
    rows=len(oddsHorses)
    o1=oddsOdds[:10*rows]
    o2=oddsOdds[10*rows:]
    try:
        hdf=pd.DataFrame(horses, columns=['course', 'marketTime', 'condition', 'distance', 'horseName','OR'])
        df=pd.DataFrame(np.hstack([np.reshape(oddsHorses,(rows,1)),np.reshape(o1,(rows,10)),np.reshape(o2,(rows,prices_count-10))]), columns=['horseName']+oddsBookies)
        df['decimalPrice']=df.loc[:, (df != "0.0").any(axis=0)].mode(axis=1).iloc[:,0]
        atr_odds.append(df)
        atrs.append(hdf.merge(df[['horseName','decimalPrice']], on='horseName', how='left'))
    except:
        print('error')
    #print(course, t, condition, distance)

#cols_base=['rid','course', 'marketTime', 'horseName', 'runners', 'ncond', 'metric', 'class', 'decimalPrice', 'age',  'RPR', 'TR', 'OR', 'weight']
pCourse = re.compile(r'<h1[^>]+>([^<]+)')
pDistance = re.compile(r'data-test-selector="RC-header__raceDistanceRound">([^<]+)')
pTime = re.compile(r'<span class="RC-courseHeader__time" data-test-selector="RC-courseHeader__time">([^<]+)')
pTitle = re.compile(r'<span data-test-selector="RC-header__raceInstanceTitle">([^<]+)')
pClass = re.compile(r'\s\((Class \d)\)\s')
pCondition = re.compile(r'Going.</div>\s+<div class="RC-headerBox__infoRow__content">([^<]+)')
pTrainer = re.compile(r'data-order-trainer="([^"]+)')
pJockey = re.compile(r'data-order-jockey="([^"]+)')
pRPR = re.compile(r'data-order-rpr="([^"]+)')
pTR = re.compile(r'data-order-ts="([^"]+)')
pOR = re.compile(r'data-order-or="([^"]+)')
pWeightSt = re.compile(r'<span class="RC-runnerWgt__carried_st">([^<]+)')
pWeightLb = re.compile(r'<span class="RC-runnerWgt__carried_lb">([^<]+)')
pAge = re.compile(r'data-order-age="([^"]+)')
pHorseName = re.compile(r'data-test-selector="RC-cardPage-runnerName">([^<]+)')

def parse_rp(html):
    course=getone(pCourse, html)
    distance=getone(pDistance, html)
    condition=getone(pCondition, html)
    rclass=getone(pClass, html)
    t=getone(pTime, html)
    title=getone(pTitle, html)
    parts=html.split('<div class="RC-runnerCardWrapper">')[1:]
    runners=len(parts)
    for part in parts:
        part=part.split('</tbody>')[0]
        horse=getone(pHorseName,part)
        trainer=getone(pTrainer,part)
        jockey=getone(pJockey,part)
        RPR=getone(pRPR,part)
        RPR='' if RPR=='-' else RPR
        TR=getone(pTR,part)
        TR='' if TR=='-' else TR
        OR=getone(pOR,part)
        OR='' if OR=='-' else OR
        weightSt=getone(pWeightSt,part)
        weightLb=getone(pWeightLb,part)
        age=getone(pAge,part)
        rps.append([course,runners,distance,condition,rclass,date.today(),title,horse,trainer,jockey,RPR,TR, weightSt,weightLb,age])
        #print(course,distance,t,date.today(),title,horse,trainer,jockey,RPR,TR, OR,weightSt,weightLb,age)



In [32]:
#load_today()
rps=[]
atrs=[]
atr_odds=[]
parse_today()
#df_atr=pd.concat(atrs)
#df_rp=pd.DataFrame(rps, columns=['course','runners','distance','condition','rclass','date','title','horseName','trainer','jockey','RPR','TR','weightSt','weightLb','age'])
#df_atr['hname']=df_atr['horseName'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')
#df_rp['hname']=df_rp['horseName'].str.lower().replace(regex=True,to_replace=r'\W',value=r'')
#df=df_rp.merge(df_atr[['marketTime', 'hname','OR','decimalPrice']], on='hname')


parse Allowance Optional Claiming_2043.html
Gulfstream
['1.73', '1.67', '1.73', '1.57', '1.73', '1.73', '1.67', '-', '1.57', '-', '5.5', '6', '5.5', '6.5', '5.5', '5.5', '6', '-', '6.5', '-', '7.5', '6.5', '6', '6', '6', '6', '6.5', '-', '6', '-', '10', '10', '10', '10', '10', '10', '10', '-', '10', '-', '10', '13', '13', '13', '13', '13', '13', '-', '13', '-', '17', '15', '13', '17', '13', '15', '15', '-', '17', '-', '41', '41', '41', '41', '41', '41', '41', '-', '41', '-', '1.73', '1.73', '-', '1.67', '1.73', '1.67', '1.67', '-', '-', '5.5', '6.5', '-', '5.5', '5.5', '5.5', '5.5', '-', '-', '6', '7', '-', '7', '6', '6.5', '6.5', '-', '-', '10', '10', '-', '10', '10', '10', '10', '-', '-', '13', '13', '-', '13', '13', '12', '13', '-', '-', '15', '17', '-', '17', '15', '13', '15', '-', '-', '41', '41', '-', '41', '41', '41', '41', '-', '-']
[1.73, 1.67, 1.73, 1.57, 1.73, 1.73, 1.67, 0, 1.57, 0, 5.5, 6.0, 5.5, 6.5, 5.5, 5.5, 6.0, 0, 6.5, 0, 7.5, 6.5, 6.0, 6.0, 6.0, 6.0, 6.5, 0, 6.0, 0, 

In [12]:
df.to_csv('data/today/{:%Y-%m-%d}.csv'.format(date.today()))

In [33]:
atrs

[       course marketTime condition distance           horseName   OR  \
 0  Gulfstream      20:43     Dirt        1m  No Getting Over Me  100   
 1  Gulfstream      20:43     Dirt        1m            Untitled  110   
 2  Gulfstream      20:43     Dirt        1m        Roman Empire   97   
 3  Gulfstream      20:43     Dirt        1m     First And Three  100   
 4  Gulfstream      20:43     Dirt        1m           Gray Beau   98   
 5  Gulfstream      20:43     Dirt        1m  Winking At Thedude   99   
 6  Gulfstream      20:43     Dirt        1m           Undaunted   84   
 
   decimalPrice  
 0          5.5  
 1         1.73  
 2          6.0  
 3         13.0  
 4         15.0  
 5         10.0  
 6         41.0  ,
         course marketTime condition distance          horseName   OR  \
 0   Gulfstream      22:17              1m 1f             Temple  108   
 1   Gulfstream      22:17              1m 1f    Bird's Eye View  110   
 2   Gulfstream      22:17              1m 1f     

In [25]:
df_atr.to_csv('data/today/atr.csv')
df_rp.to_csv('data/today/rp.csv')

In [23]:
df_rp

,course,runners,distance,condition,rclass,date,title,horseName,trainer,jockey,RPR,TR,weightSt,weightLb,age,hname
0,Morphettville,15,5½f,Good,,2020-04-18,Terry Howe Printing Handicap (2yo) (Turf),Yule My Hero,Peter Jolly,Caitlin Jones,,,9,6,2,yulemyhero
1,Morphettville,15,5½f,Good,,2020-04-18,Terry Howe Printing Handicap (2yo) (Turf),Extra Time,Leon Macdonald &amp; Andrew Gluyas,Jason Holder,,,9,5,2,extratime
2,Morphettville,15,5½f,Good,,2020-04-18,Terry Howe Printing Handicap (2yo) (Turf),Beau Rossa,Will Clarken,Todd Pannell,,,9,4,2,beaurossa
3,Morphettville,15,5½f,Good,,2020-04-18,Terry Howe Printing Handicap (2yo) (Turf),Regal Triumph,Sam Burford,Shayne Cahill,,,9,4,2,regaltriumph
4,Morphettville,15,5½f,Good,,2020-04-18,Terry Howe Printing Handicap (2yo) (Turf),Theresabearinthere,Richard &amp; Chantelle Jolly,Lachlan Neindorf,,,9,4,2,theresabearinthere
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,Oaklawn Park,12,6f,Fast,,2020-04-18,Allowance Optional Claiming Race (4yo+ Fillies...,Heavens Whisper,Paul E Holthus,Joseph Talamo,,,8,6,5,heavenswhisper
400,Oaklawn Park,12,6f,Fast,,2020-04-18,Allowance Optional Claiming Race (4yo+ Fillies...,Mt. Brave,Steven Asmussen,Ricardo Santana Jr,,,8,9,4,mtbrave
401,Oaklawn Park,12,6f,Fast,,2020-04-18,Allowance Optional Claiming Race (4yo+ Fillies...,Take Charge Angel,Ben Colebrook,Corey J Lanerie,,,8,9,4,takechargeangel
402,Oaklawn Park,12,6f,Fast,,2020-04-18,Allowance Optional Claiming Race (4yo+ Fillies...,Raintree Starlet,Brian Williamson,Miguel Mena,,,8,6,4,raintreestarlet
